<a href="https://colab.research.google.com/github/ImanAyaz10/waste-classification-project-SA-AI/blob/main/week12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os


Mounted at /content/drive


In [15]:
model_path = "/content/drive/MyDrive/mobilenet_week10.h5"
model = tf.keras.models.load_model(model_path)
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,083 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

 Optimizer params: 2 (12.00 B)

In [16]:
test_dir = "/content/drive/MyDrive/datasets/DATASET/dataset/DATASET/TEST"

cls = os.listdir(test_dir)[0]
img_name = os.listdir(os.path.join(test_dir, cls))[0]
img_path = os.path.join(test_dir, cls, img_name)

img = tf.keras.preprocessing.image.load_img(img_path, target_size=(150,150))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

print("Image loaded:", img_array.shape)


Image loaded: (1, 150, 150, 3)


In [17]:
_ = model(img_array)
print("Model initialized")


Model initialized


In [18]:
base_model = model.layers[0]

for layer in reversed(base_model.layers):
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv_layer = layer
        break

print("Last conv layer:", last_conv_layer.name)


Last conv layer: Conv_1


In [26]:
@tf.function
def gradcam(img_array):
    grad_model = tf.keras.models.Model(
        model.input,
        [last_conv_layer.output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, 0]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))

    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(conv_outputs * pooled_grads, axis=-1)

    heatmap = tf.maximum(heatmap, 0)
    heatmap /= tf.reduce_max(heatmap)
    return heatmap


In [27]:
heatmap = gradcam(img_array).numpy()

plt.imshow(heatmap)
plt.colorbar()
plt.title("Grad-CAM Heatmap")
plt.show()


AttributeError: in user code:

    File "/tmp/ipython-input-3765641605.py", line 3, in gradcam  *
        grad_model = tf.keras.models.Model(
    File "/usr/local/lib/python3.12/dist-packages/keras/src/ops/operation.py", line 276, in input
        return self._get_node_attribute_at_index(0, "input_tensors", "input")
    File "/usr/local/lib/python3.12/dist-packages/keras/src/ops/operation.py", line 307, in _get_node_attribute_at_index
        raise AttributeError(

    AttributeError: The layer sequential has never been called and thus has no defined input.


In [ ]:
img_cv = cv2.imread(img_path)
img_cv = cv2.resize(img_cv, (150,150))

heatmap_resized = cv2.resize(heatmap, (150,150))
heatmap_resized = np.uint8(255 * heatmap_resized)
heatmap_resized = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)

overlay = cv2.addWeighted(img_cv, 0.6, heatmap_resized, 0.4, 0)

plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Week 12 – Grad-CAM Result")
plt.show()
